In [2]:
#LightGBM 
#장점
#XGBoost 보다도 빠르고 높은 정확도를 보여주는 경우
#예측에 영향을 미친 변수의 중요도를 확인할 수 있음
#변수의 종류가 많고 데이터가 클수록 상대적으로 뛰어난 성능을 보여줌

#단점
#복잡한 모델인 만큼 해석이 어려움이 있음
#하이퍼파라미터 튜닝이 까다롭다.

#유용한 곳
#연속형, 범주형 모두 사용가능

#카드 거래 내역 데이터셋을 이용하여 이상거래를 예측

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

file_url='C://pythontest/KStudy/ML스터디/fraud.csv'
data=pd.read_csv(file_url)

In [42]:
data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [43]:
data.info()

# trans_date_trans_time  거래시간 
#   cc_num                 카드번호  
#   merchant               거래상점
#   category               거래상점의 범주(애완용품, 여행, 엔터테인멈트 등)
#   amt                    거래금액
#   first                  이름 성
#   last                   이름
#   gender                 성별
#   street                 고객 거주지 정보
#   city                   고객 거주지 도시
#  state                   고객 거주지 구
#  zip                     고객 거주지 우편번호 
#  lat                     고객 주소에 대한 위도
#  long                    고객 주소에 대한 경도
#  city_pop                도시 인구수
#   job                    직업
#  dob                     생년월일
#  trans_num               거래번호 
#  unix_time               거래시간
#  merch_lat               상점 위치에 대한 위도
#  merch_long              상점 위치에 대한 경도
#  is_fraud                사기거래여부


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   trans_date_trans_time  57 non-null     object 
 1   cc_num                 57 non-null     int64  
 2   merchant               57 non-null     object 
 3   category               57 non-null     object 
 4   amt                    57 non-null     float64
 5   first                  57 non-null     object 
 6   last                   57 non-null     object 
 7   gender                 57 non-null     object 
 8   street                 57 non-null     object 
 9   city                   57 non-null     object 
 10  state                  57 non-null     object 
 11  zip                    57 non-null     int64  
 12  lat                    57 non-null     float64
 13  long                   57 non-null     float64
 14  city_pop               57 non-null     int64  
 15  job     

In [44]:
data.info(show_counts=True) #non-null count 정보 보이게

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   trans_date_trans_time  57 non-null     object 
 1   cc_num                 57 non-null     int64  
 2   merchant               57 non-null     object 
 3   category               57 non-null     object 
 4   amt                    57 non-null     float64
 5   first                  57 non-null     object 
 6   last                   57 non-null     object 
 7   gender                 57 non-null     object 
 8   street                 57 non-null     object 
 9   city                   57 non-null     object 
 10  state                  57 non-null     object 
 11  zip                    57 non-null     int64  
 12  lat                    57 non-null     float64
 13  long                   57 non-null     float64
 14  city_pop               57 non-null     int64  
 15  job     

In [45]:
round(data.describe(),2)

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,5.700000e+01,57.00,57.00,57.00,57.00,57.00,5.700000e+01,57.00,57.00,57.0
mean,5.546785e+17,82.45,49443.70,38.80,-92.18,65301.11,1.325377e+09,38.82,-92.33,0.0
std,1.493264e+18,78.73,28975.98,5.39,16.57,194772.63,7.627800e+02,5.49,16.57,0.0
min,6.049559e+10,2.55,1844.00,20.08,-155.49,46.00,1.325376e+09,19.56,-156.05,0.0
25%,1.800678e+14,24.74,24433.00,36.08,-99.50,1038.00,1.325377e+09,36.29,-99.82,0.0
50%,2.348245e+15,63.07,48174.00,39.76,-88.22,3285.00,1.325377e+09,40.02,-88.52,0.0
75%,4.642255e+15,102.13,73564.00,42.22,-79.46,19685.00,1.325378e+09,42.24,-78.68,0.0
max,4.956829e+18,341.67,99160.00,48.89,-71.18,1263321.00,1.325378e+09,49.16,-71.22,0.0


In [ ]:
# amt,city_pop 에 이상치가 있다.
# is_fraud 의 mean 값이 0.01 이면 확률 1% 너무 낮습니다. 이럴 때는 정확도가 99% 라도 높아도 좋은 모델이라고 보장할 수는 없습니다.

In [46]:
data.drop(['first','last','street','city','state','zip','trans_num','unix_time','job'],axis=1, inplace=True)

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   trans_date_trans_time  57 non-null     object 
 1   cc_num                 57 non-null     int64  
 2   merchant               57 non-null     object 
 3   category               57 non-null     object 
 4   amt                    57 non-null     float64
 5   gender                 57 non-null     object 
 6   lat                    57 non-null     float64
 7   long                   57 non-null     float64
 8   city_pop               57 non-null     int64  
 9   dob                    57 non-null     object 
 10  merch_lat              57 non-null     float64
 11  merch_long             57 non-null     float64
 12  is_fraud               57 non-null     int64  
dtypes: float64(5), int64(3), object(5)
memory usage: 5.9+ KB


In [48]:
data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,36.0788,-81.1781,3495,1988-03-09,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,42.1808,-112.2620,4154,1962-01-19,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,46.2306,-112.1138,1939,1967-01-12,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,38.4207,-79.4629,99,1986-03-28,38.674999,-78.632459,0


In [49]:
data['trans_date_trans_time']=pd.to_datetime(data['trans_date_trans_time'])
#날짜 형식으로 변환

In [ ]:
#이상거래 감지의 기본적인 아이디어는 해당 고객의 기존 거래 패턴에서 벗어나는 경우 감지하는 것, 
#예를 들어 서울에서 카드를 사용하던 사람이 갑짜기 런던에서 고액으로 물건을 구매한다든가. 
#한번도 주류매장에서 이용한 적이 없는데 주류 매장에서 고액의 결재가 일어났을 경우 이상거래가 감지되면 해당 결제를 잠시 보류하고 문자/메일로 확인 받는 시스템입니다.

#Z 점수로 이상거래를 확인하기 위하여 카드 변호별 평균과 표준편차를 구한다. Z점수=x-평균/표준편차
#agg 여러개 함수를 동시에 처리 가능 #data.groupby('cc_num').agg(['mean','std'])['amt'] 
#cc_num 으로 그룹핑하여 amt 의 평균과 표준 편차 동시에 구하여 amt_info에 정보를 저장합니다.

In [50]:
amt_info=data.groupby('cc_num').agg(['mean','std'])['amt'].reset_index()

C:\Users\minjung.bae\AppData\Local\Temp\ipykernel_28028\602477151.py:1: FutureWarning: ['trans_date_trans_time', 'merchant', 'category', 'gender', 'dob'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  amt_info=data.groupby('cc_num').agg(['mean','std'])['amt'].reset_index()


In [51]:
amt_info.head()

,cc_num,mean,std
0,60495593109,122.86,NaN
1,630412733309,66.21,NaN
2,630423337322,107.23,NaN
3,630441765090,9.55,NaN
4,676372984911,53.60,NaN


In [52]:
data=data.merge(amt_info,on='cc_num',how='left')
#cc_num 을 키 값으로 기존 데이터에 이 데이터를 left join 으로 붙혀 원하는 형태의 데이터를 얻을 수 있습니다.

In [53]:
data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,mean,std
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,36.0788,-81.1781,3495,1988-03-09,36.011293,-82.048315,0,4.97,NaN
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,107.23,NaN
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,42.1808,-112.2620,4154,1962-01-19,43.150704,-112.154481,0,220.11,NaN
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,46.2306,-112.1138,1939,1967-01-12,47.034331,-112.561071,0,45.00,NaN
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,38.4207,-79.4629,99,1986-03-28,38.674999,-78.632459,0,41.96,NaN


In [54]:
#z점수 내기
data['amt_z_score']=(data['amt']-data['mean'])/data['std']

In [55]:
data[['amt','mean','std','amt_z_score']].head()

,amt,mean,std,amt_z_score
0,4.97,4.97,NaN,NaN
1,107.23,107.23,NaN,NaN
2,220.11,220.11,NaN,NaN
3,45.00,45.00,NaN,NaN
4,41.96,41.96,NaN,NaN


In [56]:
data.drop(['mean','std'],axis=1,inplace=True)

In [57]:
#개인마다 어떤 범주에 얼마만큼의 금액을 쓰는지 다를것이다.
category_info=data.groupby(['cc_num','category']).agg(['mean','std'])['amt'].reset_index()
#cc_num,category 기준으로 amt의 평균과 표준편차 계산

C:\Users\minjung.bae\AppData\Local\Temp\ipykernel_28028\3640278815.py:2: FutureWarning: ['trans_date_trans_time', 'merchant', 'gender', 'dob'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  category_info=data.groupby(['cc_num','category']).agg(['mean','std'])['amt'].reset_index()


In [58]:
category_info.head()

,cc_num,category,mean,std
0,60495593109,food_dining,122.86,NaN
1,630412733309,shopping_pos,66.21,NaN
2,630423337322,grocery_pos,107.23,NaN
3,630441765090,shopping_pos,9.55,NaN
4,676372984911,gas_transport,53.60,NaN


In [59]:
data=data.merge(category_info, on =['cc_num','category'],how='left')

In [60]:
data['cat_z_score']=(data['amt']-data['mean'])/data['std']

In [61]:
data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z_score,mean,std,cat_z_score
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,36.0788,-81.1781,3495,1988-03-09,36.011293,-82.048315,0,NaN,4.97,NaN,NaN
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,NaN,107.23,NaN,NaN
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,42.1808,-112.2620,4154,1962-01-19,43.150704,-112.154481,0,NaN,220.11,NaN,NaN
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,46.2306,-112.1138,1939,1967-01-12,47.034331,-112.561071,0,NaN,45.00,NaN,NaN
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,38.4207,-79.4629,99,1986-03-28,38.674999,-78.632459,0,NaN,41.96,NaN,NaN


In [62]:
data.drop(['mean','std'],axis=1,inplace=True)

In [63]:
#거리로 z 점수 체크
#pip install geopy
import geopy.distance

In [64]:
data['merch_coord']=pd.Series(zip(data['merch_lat'],data['merch_long']))

data['cust_coord']=pd.Series(zip(data['lat'],data['long']))

In [66]:
data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z_score,cat_z_score,merch_coord,cust_coord
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,36.0788,-81.1781,3495,1988-03-09,36.011293,-82.048315,0,NaN,NaN,"(36.011293, -82.048315)","(36.0788, -81.1781)"
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,NaN,NaN,"(49.159047, -118.186462)","(48.8878, -118.2105)"
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,42.1808,-112.2620,4154,1962-01-19,43.150704,-112.154481,0,NaN,NaN,"(43.150704, -112.154481)","(42.1808, -112.262)"
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,46.2306,-112.1138,1939,1967-01-12,47.034331,-112.561071,0,NaN,NaN,"(47.034331, -112.561071)","(46.2306, -112.1138)"
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,38.4207,-79.4629,99,1986-03-28,38.674999,-78.632459,0,NaN,NaN,"(38.674999, -78.632459)","(38.4207, -79.4629)"


In [74]:
data['distance']=data.apply(lambda x: geopy.distance.distance(x['merch_coord'],x['cust_coord']).km, axis=1)

In [76]:
#코드가 실행된 시간을 측정하기
import time

start_time=time.time()
data['distance']=data.apply(lambda x: geopy.distance.distance(x['merch_coord'],x['cust_coord']).km, axis=1)
end_time=time.time()
print(end_time-start_time)

0.010970592498779297


In [77]:
data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z_score,cat_z_score,merch_coord,cust_coord,distance
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,36.0788,-81.1781,3495,1988-03-09,36.011293,-82.048315,0,NaN,NaN,"(36.011293, -82.048315)","(36.0788, -81.1781)",78.773821
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,NaN,NaN,"(49.159047, -118.186462)","(48.8878, -118.2105)",30.216618
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,42.1808,-112.2620,4154,1962-01-19,43.150704,-112.154481,0,NaN,NaN,"(43.150704, -112.154481)","(42.1808, -112.262)",108.102912
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,46.2306,-112.1138,1939,1967-01-12,47.034331,-112.561071,0,NaN,NaN,"(47.034331, -112.561071)","(46.2306, -112.1138)",95.685115
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,38.4207,-79.4629,99,1986-03-28,38.674999,-78.632459,0,NaN,NaN,"(38.674999, -78.632459)","(38.4207, -79.4629)",77.702395


In [81]:

distance_info=data.groupby('cc_num').agg(['mean','std'])['distance'].reset_index()



C:\Users\minjung.bae\AppData\Local\Temp\ipykernel_28028\2456148903.py:1: FutureWarning: ['trans_date_trans_time', 'merchant', 'category', 'gender', 'dob', 'merch_coord', 'cust_coord'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  distance_info=data.groupby('cc_num').agg(['mean','std'])['distance'].reset_index()


In [84]:
data=data.merge(distance_info,on ='cc_num',how='left')


In [85]:
data['distance_z_score']=(data['distance']-data['mean'])/data['std']
data.drop(['mean','std'],axis=1,inplace=True)
data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z_score,cat_z_score,merch_coord,cust_coord,distance,distance_z_score
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,36.0788,-81.1781,3495,1988-03-09,36.011293,-82.048315,0,NaN,NaN,"(36.011293, -82.048315)","(36.0788, -81.1781)",78.773821,NaN
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,NaN,NaN,"(49.159047, -118.186462)","(48.8878, -118.2105)",30.216618,NaN
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,42.1808,-112.2620,4154,1962-01-19,43.150704,-112.154481,0,NaN,NaN,"(43.150704, -112.154481)","(42.1808, -112.262)",108.102912,NaN
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,46.2306,-112.1138,1939,1967-01-12,47.034331,-112.561071,0,NaN,NaN,"(47.034331, -112.561071)","(46.2306, -112.1138)",95.685115,NaN
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,38.4207,-79.4629,99,1986-03-28,38.674999,-78.632459,0,NaN,NaN,"(38.674999, -78.632459)","(38.4207, -79.4629)",77.702395,NaN


In [87]:
#나이 구하기
data['age']=2023-pd.to_datetime(data['dob']).dt.year

In [88]:
data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z_score,cat_z_score,merch_coord,cust_coord,distance,distance_z_score,age
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,36.0788,-81.1781,3495,1988-03-09,36.011293,-82.048315,0,NaN,NaN,"(36.011293, -82.048315)","(36.0788, -81.1781)",78.773821,NaN,35
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0,NaN,NaN,"(49.159047, -118.186462)","(48.8878, -118.2105)",30.216618,NaN,45
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,42.1808,-112.2620,4154,1962-01-19,43.150704,-112.154481,0,NaN,NaN,"(43.150704, -112.154481)","(42.1808, -112.262)",108.102912,NaN,61
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,46.2306,-112.1138,1939,1967-01-12,47.034331,-112.561071,0,NaN,NaN,"(47.034331, -112.561071)","(46.2306, -112.1138)",95.685115,NaN,56
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,38.4207,-79.4629,99,1986-03-28,38.674999,-78.632459,0,NaN,NaN,"(38.674999, -78.632459)","(38.4207, -79.4629)",77.702395,NaN,37


In [89]:
data.drop(['cc_num','lat','long','merch_lat','merch_long','dob','merch_coord','cust_coord'],axis=1,inplace=True)
data.head()

,trans_date_trans_time,merchant,category,amt,gender,city_pop,is_fraud,amt_z_score,cat_z_score,distance,distance_z_score,age
0,2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,3495,0,NaN,NaN,78.773821,NaN,35
1,2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,149,0,NaN,NaN,30.216618,NaN,45
2,2019-01-01 00:00:51,fraud_Lind-Buckridge,entertainment,220.11,M,4154,0,NaN,NaN,108.102912,NaN,61
3,2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,1939,0,NaN,NaN,95.685115,NaN,56
4,2019-01-01 00:03:06,fraud_Keeling-Crist,misc_pos,41.96,M,99,0,NaN,NaN,77.702395,NaN,37


In [91]:
#더미 변수로 변환
data=pd.get_dummies(data,columns=['category','gender'],drop_first=True)

In [93]:
data.set_index('trans_date_trans_time',inplace=True) #인덱스 재설정
data.head()


,merchant,amt,city_pop,is_fraud,amt_z_score,cat_z_score,distance,distance_z_score,age,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_misc_net,category_misc_pos,category_shopping_net,category_shopping_pos,gender_M
trans_date_trans_time,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",4.97,3495,0,NaN,NaN,78.773821,NaN,35,0,0,0,0,1,0,0,0,0
2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",107.23,149,0,NaN,NaN,30.216618,NaN,45,0,0,0,1,0,0,0,0,0
2019-01-01 00:00:51,fraud_Lind-Buckridge,220.11,4154,0,NaN,NaN,108.102912,NaN,61,0,0,0,0,0,0,0,0,1
2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",45.00,1939,0,NaN,NaN,95.685115,NaN,56,0,1,0,0,0,0,0,0,1
2019-01-01 00:03:06,fraud_Keeling-Crist,41.96,99,0,NaN,NaN,77.702395,NaN,37,0,0,0,0,0,1,0,0,1


In [158]:
train=data[data.index<'2019-01-01 00:10:00'] #훈련셋 설정
test=data[data.index>='2019-01-01 00:10:00'] #시험셋 설정

In [159]:
len(test) / len(data) #시험셋과 비율 확인

0.7192982456140351

In [160]:
#훈련셋 시험셋을 나눈다.
from sklearn.model_selection import train_test_split


In [163]:
X_train, X_test, y_train, y_test=train_test_split(data.drop('is_fraud',axis=1),data['is_fraud'],test_size=0.2,random_state=100)

In [162]:
X_train=train.drop('is_fraud',axis=1)
X_test=test.drop('is_fraud',axis=1)


In [133]:
y_train=train['is_fraud']
y_test=test['is_fraud']

In [164]:
import lightgbm as lgb

In [165]:
model_1=lgb.LGBMClassifier(random_state=100)


In [150]:
data.drop(['merchant'],axis=1,inplace=True)

In [166]:
train

,amt,city_pop,is_fraud,amt_z_score,cat_z_score,distance,distance_z_score,age,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_misc_net,category_misc_pos,category_shopping_net,category_shopping_pos,gender_M
trans_date_trans_time,,,,,,,,,,,,,,,,,
2019-01-01 00:00:18,4.97,3495,0,NaN,NaN,78.773821,NaN,35,0,0,0,0,1,0,0,0,0
2019-01-01 00:00:44,107.23,149,0,NaN,NaN,30.216618,NaN,45,0,0,0,1,0,0,0,0,0
2019-01-01 00:00:51,220.11,4154,0,NaN,NaN,108.102912,NaN,61,0,0,0,0,0,0,0,0,1
2019-01-01 00:01:16,45.00,1939,0,NaN,NaN,95.685115,NaN,56,0,1,0,0,0,0,0,0,1
2019-01-01 00:03:06,41.96,99,0,NaN,NaN,77.702395,NaN,37,0,0,0,0,0,1,0,0,1
2019-01-01 00:04:08,94.63,2158,0,NaN,NaN,86.097358,NaN,62,0,1,0,0,0,0,0,0,0
2019-01-01 00:04:42,44.54,2691,0,-0.707107,NaN,118.094855,0.707107,30,0,0,1,0,0,0,0,0,0
2019-01-01 00:05:08,71.65,6018,0,NaN,NaN,12.754714,NaN,76,0,1,0,0,0,0,0,0,1
2019-01-01 00:05:18,4.27,1472,0,NaN,NaN,25.333883,NaN,82,0,0,0,0,0,1,0,0,0


In [157]:
X_train

,merchant,amt,city_pop,amt_z_score,cat_z_score,distance,distance_z_score,age,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_misc_net,category_misc_pos,category_shopping_net,category_shopping_pos,gender_M
trans_date_trans_time,,,,,,,,,,,,,,,,,
2019-01-01 00:00:18,"fraud_Rippin, Kub and Mann",4.97,3495,NaN,NaN,78.773821,NaN,35,0,0,0,0,1,0,0,0,0
2019-01-01 00:00:44,"fraud_Heller, Gutmann and Zieme",107.23,149,NaN,NaN,30.216618,NaN,45,0,0,0,1,0,0,0,0,0
2019-01-01 00:00:51,fraud_Lind-Buckridge,220.11,4154,NaN,NaN,108.102912,NaN,61,0,0,0,0,0,0,0,0,1
2019-01-01 00:01:16,"fraud_Kutch, Hermiston and Farrell",45.00,1939,NaN,NaN,95.685115,NaN,56,0,1,0,0,0,0,0,0,1
2019-01-01 00:03:06,fraud_Keeling-Crist,41.96,99,NaN,NaN,77.702395,NaN,37,0,0,0,0,0,1,0,0,1
2019-01-01 00:04:08,"fraud_Stroman, Hudson and Erdman",94.63,2158,NaN,NaN,86.097358,NaN,62,0,1,0,0,0,0,0,0,0
2019-01-01 00:04:42,fraud_Rowe-Vandervort,44.54,2691,-0.707107,NaN,118.094855,0.707107,30,0,0,1,0,0,0,0,0,0
2019-01-01 00:05:08,fraud_Corwin-Collins,71.65,6018,NaN,NaN,12.754714,NaN,76,0,1,0,0,0,0,0,0,1
2019-01-01 00:05:18,fraud_Herzog Ltd,4.27,1472,NaN,NaN,25.333883,NaN,82,0,0,0,0,0,1,0,0,0


In [168]:
model_1.fit(X_train,y_train)
model_1

LGBMClassifier(random_state=100)

In [170]:
pred_1=model_1.predict(X_test)

In [171]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,roc_auc_score


In [172]:
accuracy_score(y_test,pred_1)

1.0

In [174]:
#혼동행렬 확인
print(confusion_matrix(y_test,pred_1))

[[12]]


In [175]:
print(classification_report(y_test,pred_1))

#정밀도(0.54) 재현율 0.59, ㄹ-1 0.56
#정밀도는 정상거래를 정상거래로 판단하는것
#불편함이 조금더 발생되더라도 이상거래를 조금더 포착하는 것이 중요하다
#즉 실제 이상거래를 얼마나 잘 예측해냈는지가 더 의미하는 재현율이 더 중요하다. 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12

    accuracy                           1.00        12
   macro avg       1.00      1.00      1.00        12
weighted avg       1.00      1.00      1.00        12



In [176]:
proba_1_1=model_1.predict_proba(X_test)

In [177]:
proba_1_1

array([[1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15]])

In [178]:
from sklearn.model_selection import RandomizedSearchCV

In [179]:
params={'n_estimators':[100,500,1000], #반복횟수
'learning_rate':[0.01,0.05,0.1,0.3], #러닝 레이트
'lambda_l1':[0,10,20,30,50], #L1정규화
'lambda_l2':[0,10,20,30,50], #L2정규화
'max_depth':[5,10,15,20], #최대길이
'subsample':[0.6,0.8,1], #서브셈플비율
}

In [180]:
model_2=lgb.LGBMClassifier(random_state=100)
rs=RandomizedSearchCV(model_2,param_distributions=params,n_iter=30,scoring='roc_auc',random_state=100,n_jobs=-1)

In [181]:
start=time.time()
rs.fit(X_train,y_train)
print(time.time()-start)

[LightGBM] [Warning] lambda_l1 is set=20, reg_alpha=0.0 will be ignored. Current value: lambda_l1=20
[LightGBM] [Warning] lambda_l2 is set=0, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0
6.5435802936553955


c:\Users\minjung.bae\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


In [182]:
rs.best_params_

{'subsample': 1,
 'n_estimators': 500,
 'max_depth': 20,
 'learning_rate': 0.1,
 'lambda_l2': 0,
 'lambda_l1': 20}

In [186]:
y_test

trans_date_trans_time
2019-01-01 00:11:14    0
2019-01-01 00:04:42    0
2019-01-01 00:30:49    0
2019-01-01 00:36:58    0
2019-01-01 00:37:18    0
2019-01-01 00:20:15    0
2019-01-01 00:06:56    0
2019-01-01 00:22:06    0
2019-01-01 00:00:18    0
2019-01-01 00:27:45    0
2019-01-01 00:40:50    0
2019-01-01 00:06:53    0
Name: is_fraud, dtype: int64

In [188]:
X_test

,amt,city_pop,amt_z_score,cat_z_score,distance,distance_z_score,age,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_misc_net,category_misc_pos,category_shopping_net,category_shopping_pos,gender_M
trans_date_trans_time,,,,,,,,,,,,,,,,
2019-01-01 00:11:14,63.07,2518,NaN,NaN,90.916639,NaN,77,1,0,0,0,0,0,0,0,1
2019-01-01 00:04:42,44.54,2691,-0.707107,NaN,118.094855,0.707107,30,0,0,1,0,0,0,0,0,0
2019-01-01 00:30:49,53.60,242,NaN,NaN,82.856315,NaN,36,0,1,0,0,0,0,0,0,0
2019-01-01 00:36:58,130.89,1686,NaN,NaN,29.193265,NaN,37,0,0,0,1,0,0,0,0,0
2019-01-01 00:37:18,103.00,969,NaN,NaN,22.724076,NaN,69,0,0,0,1,0,0,0,0,0
2019-01-01 00:20:15,64.09,31515,NaN,NaN,22.322745,NaN,52,0,0,0,1,0,0,0,0,1
2019-01-01 00:06:56,71.22,341043,NaN,NaN,44.606380,NaN,34,0,0,0,1,0,0,0,0,1
2019-01-01 00:22:06,90.22,4878,NaN,NaN,82.244106,NaN,57,0,0,0,1,0,0,0,0,1
2019-01-01 00:00:18,4.97,3495,NaN,NaN,78.773821,NaN,35,0,0,0,0,1,0,0,0,0


In [187]:
rs_proba=rs.predict_proba(X_test)
roc_auc_score(y_test,rs_proba[:,1])

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [191]:
proba_1=model_1.predict_proba(X_test)

In [192]:
proba_1

array([[1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15],
       [1.e+00, 1.e-15]])

In [193]:
proba_1[:,1]

array([1.e-15, 1.e-15, 1.e-15, 1.e-15, 1.e-15, 1.e-15, 1.e-15, 1.e-15,
       1.e-15, 1.e-15, 1.e-15, 1.e-15])

In [194]:
proba_1=proba_1[:,1]

In [195]:
proba_int1=(proba_1>0.2).astype('int')
proba_int2=(proba_1>0.8).astype('int')

In [196]:
print(confusion_matrix(y_test,proba_int1))

[[12]]


In [197]:
print(classification_report(y_test,proba_int1))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12

    accuracy                           1.00        12
   macro avg       1.00      1.00      1.00        12
weighted avg       1.00      1.00      1.00        12



In [198]:
print(confusion_matrix(y_test,proba_int2))

[[12]]


In [199]:
print(classification_report(y_test,proba_int2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12

    accuracy                           1.00        12
   macro avg       1.00      1.00      1.00        12
weighted avg       1.00      1.00      1.00        12

